In [100]:
import os, sys, pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
import hvplot.pandas
from bokeh.io import curdoc
import datetime


curdoc().theme = "dark_minimal"

In [25]:
grade_conversion = {
    "6B+": 4,
    "6C" : 5,
    "6C+": 5.5,
    "7A" : 6,
    "7A+": 7,
    "7B" : 8,
    "7B+": 8.5
}

In [84]:
df = pd.read_csv("full_formatted_data_231213.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       491 non-null    object 
 1   name       491 non-null    object 
 2   grade      491 non-null    object 
 3   setter     491 non-null    object 
 4   mygrade    491 non-null    object 
 5   attempts   491 non-null    int64  
 6   ticked     491 non-null    bool   
 7   comment    3 non-null      object 
 8   grade_v    491 non-null    float64
 9   mygrade_v  491 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(6)
memory usage: 35.1+ KB


In [128]:
# Group by problem and report number of tries, if it is ticked or not, number of days, setter
from collections import namedtuple

def _groupby(raw_df):
    
    def count_days(dt_series):
        return len(set(dt_series))
    
    df = raw_df[:]

    df.loc[:, "date"] = pd.to_datetime(df.date, format="%Y-%m-%d").dt.date

    df = raw_df.groupby(["name","grade_v", "setter"], as_index=False).aggregate(
        total_attempts = ("attempts", np.sum),
        total_days = ("date", count_days),
        ticked = ("ticked", np.any),
        last_tried = ("date", max),
        first_tried = ("date", min)
    )

    # TODO: Not sure how to do aggregating functions on groupby using multiple columns
    # e.g. passing in both 'date' and 'ticked' as inputs. until then.. using a loop and merge

    detailed_stats = []

    DateStats = namedtuple(
        "DateStats", 
        "name \
            first_ticked \
                attempts_to_send \
                    days_to_send \
                        num_repeats"
    )

    for n in df.name:
        first_ticked = np.nan
        tdf = raw_df[raw_df.name == n]
        ticked_sessions = tdf[tdf.ticked]
        
        try:
            first_ticked = ticked_sessions.date.min()
            attempts_to_send = tdf.loc[tdf.date<=first_ticked, "attempts"].sum()
            days_to_send = tdf.loc[tdf.date<=first_ticked].__len__()
        
        except Exception as e:
            print(e)
            first_ticked = np.nan
            attempts_to_send = np.nan
            days_to_send = np.nan

        num_repeats = ticked_sessions.__len__()
        
        detailed_stats.append(
            DateStats(n, first_ticked, attempts_to_send, days_to_send, num_repeats)
        )

    detailed_stats_df = pd.DataFrame(detailed_stats, columns=DateStats._fields)

    merged = pd.merge(left=df, right=detailed_stats_df, how='left', on='name')

    return merged


def current_projects_ranked(raw_df, as_table=True):
    df = _groupby(raw_df)
    df = df[~df.ticked].sort_values(
        ["total_days", "total_attempts"], 
        ascending=False
    )
    df = df.drop(columns=['num_repeats', 'attempts_to_send', 'days_to_send', 'first_ticked', 'ticked'])
    
    df['last_tried_days'] = (datetime.date.today() - df.last_tried)
    df['span_days'] = (df.last_tried - df.first_tried)

    if as_table:
        return df.hvplot.table(sortable=True, selectable=True)

    return df

def ticked_climbs_ranked(
        raw_df,
        topk=10, 
        by=["attempts_to_send", "num_repeats"], 
        ascending=[False, True]
        ):
    # By default returns top ticked climbs ranked by most attempts needed, then by least repeats
    # good for choosing as repeats
    df = _groupby(raw_df)
    return df[df.ticked].sort_values(by=by, ascending=ascending).drop(columns='ticked')[:10]

# def ticked_climbs_ranked(raw_df, topk=10, by='attempts'):
    
#     # Get top-k list of climbs by difficulty
#     topk_climbs = raw_df.groupby(["name"], as_index=False).aggregate(
#         difficulty = ("grade_v", np.max)
#     ).loc[:topk, "name"]

#     topk_climbs = set(topk_climbs.tolist())
#     return topk_climbs

#     return df
    # return df[df.ticked].hvplot.table(sortable=True, selectable=True)



# df2[~df2.ticked].groupby("setter")




In [130]:
grp = _groupby(df)

In [131]:
grp

,name,grade_v,setter,total_attempts,total_days,ticked,last_tried,first_tried,first_ticked,attempts_to_send,days_to_send,num_repeats
0,#2 KH,5.5,Kyle Hilton,3,2,True,2022-10-06,2022-09-01,2022-09-01,2,1,2
1,(*)(*)FAKE,4.0,Mystery Frog Man,1,1,True,2023-05-30,2023-05-30,2023-05-30,1,1,1
2,1,4.0,Hugo Hoyer,1,1,True,2023-11-26,2023-11-26,2023-11-26,1,1,1
3,1817,5.0,Randall Baum,3,1,False,2023-06-13,2023-06-13,NaN,0,0,0
4,1K,4.0,Bryant Noble,1,1,True,2023-11-28,2023-11-28,2023-11-28,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
232,¡APUNT****!,4.0,AJ Rimes,1,1,True,2023-12-12,2023-12-12,2023-12-12,1,1,1
233,紙一重 -KAMI HITOE-,5.5,seishirou shirai,6,1,True,2022-09-01,2022-09-01,2022-09-01,6,2,2
234,課題2,5.5,atsushi takizawa,3,1,True,2023-02-16,2023-02-16,2023-02-16,3,1,1
235,課題24,4.0,atsushi takizawa,1,1,True,2023-01-24,2023-01-24,2023-01-24,1,1,1


In [119]:
current_projects_ranked(df, as_table=False)

,name,grade_v,setter,total_attempts,total_days,last_tried,first_tried,last_tried_days,span_days
100,GRIFETE HANDS,5.0,edu moya,15,7,2023-12-02,2022-09-29,13 days,429 days
71,EASY DOES IT,4.0,Reuben Mardan,15,5,2023-10-17,2023-02-11,59 days,248 days
135,MOONING,5.0,levi jackson,14,5,2023-11-09,2023-03-05,36 days,249 days
101,HARD TIMES,6.0,Ben Moon,13,4,2022-12-29,2022-11-24,351 days,35 days
75,ENTER SANDBAG,5.5,Christian Thorjussen,8,4,2023-02-28,2023-01-31,290 days,28 days
128,MAFFEI,5.0,Riccardo Caprasecca,7,3,2023-12-05,2023-11-30,10 days,5 days
102,HATHA YOGA,4.0,janezkranjski,6,3,2023-10-07,2022-11-29,69 days,312 days
127,LOW CARB VAR,5.0,Ben Moon,7,2,2023-11-28,2023-11-16,17 days,12 days
59,DANIELSAN,5.5,Jack Folkes,6,2,2023-07-18,2023-06-20,150 days,28 days
98,GRAB THE D,5.0,Martin Heald,6,2,2023-06-15,2023-06-13,183 days,2 days


In [129]:
ticked_climbs_ranked(df)

,name,grade_v,setter,total_attempts,total_days,last_tried,first_tried,first_ticked,attempts_to_send,days_to_send,num_repeats
23,ASTRO CREEP,4.0,Jonny Summers,46,7,2023-11-21,2022-11-17,2023-11-21,46,7,1
153,POGO,6.0,luke hutcheson,16,4,2023-07-13,2023-03-23,2023-06-17,14,4,2
113,IMPROVVISAZIONE,6.0,Jean-Pierre Bovo,13,3,2022-12-08,2022-11-17,2022-12-08,13,3,1
140,NORTHSIDE,6.0,Matt Mcnair,13,4,2023-02-21,2022-12-08,2023-02-21,13,4,1
175,SHAKEDOWN STREET,5.5,Matt Poiesz,13,4,2022-11-15,2022-10-26,2022-11-15,13,4,1
134,MOON CASTLE,5.0,Ben Moon,12,5,2023-02-26,2022-09-10,2023-02-26,12,5,1
213,VIVA,4.0,Hakan Ertekin,12,3,2023-01-19,2022-12-15,2023-01-19,12,4,1
161,RAINY DAY,4.0,Chris Tibbals,12,3,2023-02-26,2023-02-16,2023-02-26,12,4,2
146,OVN,5.0,Jørgen Bryn Henriksen,10,5,2022-11-15,2022-10-01,2022-11-15,10,5,1
177,SISÚ,5.0,edu moya,10,4,2023-05-06,2023-03-02,2023-05-06,10,4,1


In [44]:
proj = current_projects_ranked(df)

In [91]:
proj

:Table   [name,grade_v,setter,total_attempts,total_days,ticked,last_tried,first_tried,date_first_ticked,attempts_to_send,days_to_send,num_repeats]

In [54]:
ticked = ticked_climbs_ranked(df)

In [55]:
ticked

{'#2 KH',
 '(*)(*)FAKE',
 '1',
 '1817',
 '1K',
 '2',
 '272966',
 '7A TOO MUCH',
 '7STÖCKE',
 'A LITTLE CROSSING',
 'AB1'}